In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [14]:
url = "https://en.wikipedia.org/wiki/List_of_female_Academy_Award_winners_and_nominees_for_non-gendered_categories"
page = requests.get( url )
soup = BeautifulSoup(page.content, 'html.parser')

<td><a href="/wiki/80th_Academy_Awards" title="80th Academy Awards">2007</a>
</td>

#### Lendo o html
##### table datacells (td)
- td[0] = título;
- td[1 + 5k] = ano;
- td[4 + 5k] = status;


In [62]:
d = {'categoria': [], 'ano': [], 'status': []}
oscar_df = pd.DataFrame(data=d)
all_tables = soup.find_all(class_='wikitable')
for table in all_tables:
	td = table.find_all('td')
	oscar_df.categoria[0] = td[0]
	break
oscar_df

,categoria,ano,status


In [58]:
table = all_tables[0]
td = table.find_all('td')
td[14].text

'Won\n'

In [4]:
categoria = soup.find(id="Best_Animated_Feature")
categoria

<span class="mw-headline" id="Best_Animated_Feature">Best Animated Feature</span>